In [1]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 96 kB/s 
     |████████████████████████████████| 6.8 MB 30.2 MB/s 
     |████████████████████████████████| 981 kB 41.6 MB/s 
     |████████████████████████████████| 263 kB 48.7 MB/s 
     |████████████████████████████████| 1.9 MB 50.0 MB/s 
     |████████████████████████████████| 1.2 MB 38.3 MB/s 
     |████████████████████████████████| 468 kB 49.5 MB/s 
     |████████████████████████████████| 895 kB 48.8 MB/s 
     |████████████████████████████████| 3.3 MB 41.6 MB/s 
  Created wheel for ktrain: filename=ktrain-0.27.2-py3-none-any.whl size=25283087 sha256=f32301facdf91e136f2d773247fbc51f20cb88124d310b8c19dd41335ab596cf
  Stored in directory: /root/.cache/pip/wheels/88/be/4a/971c83a380a40f12e877f643ca1b94dc65f528f94c88dbcff7
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=c0800fed8ca78b1ea9dbc18f6399e22b68912462ac37730f7d8bccea132fa73b
  Stored in directory: /root/.cache/pip/wheels/f5/ac/f1/4e13d7aff0

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID';
os.environ['CUDA_VISIBLE_DEVICES']='0';

In [3]:
import ktrain
from ktrain import text
from sklearn.datasets import fetch_20newsgroups

In [4]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med', 'rec.sport.baseball']

In [5]:
train = fetch_20newsgroups(subset='train',categories=categories,shuffle=True,random_state=0)

In [6]:
test = fetch_20newsgroups(subset='test',categories=categories,shuffle=True,random_state=0)

In [7]:
test.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [8]:
test.target

array([0, 4, 2, ..., 2, 3, 0])

In [9]:
test.target_names

['alt.atheism',
 'comp.graphics',
 'rec.sport.baseball',
 'sci.med',
 'soc.religion.christian']

In [10]:
x_train = train.data
y_train = train.target

In [11]:
x_test = test.data
y_test = test.target

### **Build ML Transformer model**

In [14]:
model_name = 'distilbert-base-uncased' # base = smaller version of the model and uncased = not case sensitive

In [15]:
trans = text.Transformer(model_name=model_name,maxlen=512,class_names=categories)

In [16]:
train_data = trans.preprocess_train(x_train, y_train)
test_data = trans.preprocess_test(x_test, y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 291
	95percentile : 820
	99percentile : 1757


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 323
	95percentile : 894
	99percentile : 2394


In [17]:
model = trans.get_classifier()

In [18]:
learner = ktrain.get_learner(model,train_data=train_data,val_data=test_data, batch_size=16)

In [19]:
learner.lr_find(show_plot=True, max_epochs=1)  # increase the epochs as required

simulating training for different learning rates... this may take a few moments...
 51/178 [=======>......................] - ETA: 3:30 - loss: 1.6026 - accuracy: 0.2316

In [20]:
learner.fit_onecycle(1e-4, 1)



begin training using onecycle policy with max lr of 0.0001...
179/179 [==============================] - 373s 2s/step - loss: 0.4958 - accuracy: 0.8490 - val_loss: 0.1722 - val_accuracy: 0.9489


In [21]:
learner

In [22]:
learner.validate(class_names=categories) # confusion matrix

              precision    recall  f1-score   support

           0       0.89      0.88      0.88       319
           1       0.97      0.96      0.97       389
           2       0.99      0.97      0.98       397
           3       0.96      0.95      0.96       396
           4       0.92      0.96      0.94       398

    accuracy                           0.95      1899
   macro avg       0.95      0.95      0.95      1899
weighted avg       0.95      0.95      0.95      1899



array([[281,   0,   0,   9,  29],
       [  9, 375,   0,   5,   0],
       [  4,   4, 387,   1,   1],
       [ 10,   5,   2, 377,   2],
       [ 13,   2,   0,   1, 382]])

In [23]:
learner.view_top_losses(n=5, preproc=trans) # determine which data is not performing good

----------
id:562 | loss:5.8 | true:rec.sport.baseball | pred:soc.religion.christian)

----------
id:311 | loss:5.69 | true:sci.med | pred:alt.atheism)

----------
id:730 | loss:5.45 | true:alt.atheism | pred:sci.med)

----------
id:787 | loss:5.33 | true:comp.graphics | pred:soc.religion.christian)

----------
id:335 | loss:5.31 | true:sci.med | pred:rec.sport.baseball)



In [24]:
x_test[311]

"From: turpin@cs.utexas.edu (Russell Turpin)\nSubject: Meaning of atheism, agnosticism  (was: Krillean Photography)\nOrganization: CS Dept, University of Texas at Austin\nLines: 23\nNNTP-Posting-Host: saltillo.cs.utexas.edu\n\n-*----\nSci.med removed from followups.  (And I do not read any of the\nother newsgroups.)\n\n-*----\nIn article <1993Apr30.170233.12510@rosevax.rosemount.com> grante@aquarius.rosemount.com (Grant Edwards) writes:\n> As a self-proclaimed atheist my position is that I _believe_ that \n> there is no god.  I don't claim to have any proof.  I interpret\n> the agnostic position as having no beliefs about god's existence.\n\nThat's fine.  These words have multiple meanings.\n\nAs a self-proclaimed atheist, I believe that *some* conceptions\nof god are inconsistent or in conflict with fact, and I lack\nbelief in other conceptions of god merely because there is no\nreason for me to believe in these.  I usually use the word\nagnostic to mean someone who believes that the 

In [25]:
predictor = ktrain.get_predictor(learner.model, preproc=trans)

In [26]:
x = ['Jesus Christ is the central figure of Christianity']

In [27]:
predictor.predict(x)

['rec.sport.baseball']